In [208]:
### To maintain the packages and the the used versions
# !pip install pipreqsnb
!pipreqsnb --force ./

pipreqs  --force ./


INFO: Not scanning for jupyter notebooks.
Please, verify manually the final list of requirements.txt to avoid possible dependency confusions.
Please, verify manually the final list of requirements.txt to avoid possible dependency confusions.
INFO: Successfully saved requirements file in ./requirements.txt


In [7]:
# pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


# Loading Data

English to Hindi translation pairs from the MUSE dataset by Meta.

In [63]:
import pandas as pd
import numpy as np

In [5]:
train_path = "data/train.txt"
test_path = "data/test.txt"

We are loading the text pairs into a pandas dataframe for easy access. Reading with Pandas was chosen over directly loading from the file as the system was unable to recognize the Devanagri script if loaded directly and was throwing errors. The dataframes are then converted into a dictionaries,

Separate train and test files were downloaded from the github repository and loaded accordingly. The train dataset contains 8704 text pairs and the test set has 2032 pairs. Sample of the dataset is also shown. Each entity of the dataframe is a string. The dataframes are then converted to dictionaries for ease of access.

In [154]:
train_df = pd.read_csv(train_path, sep = '\t', header = None)
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8704 entries, 0 to 8703
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       8704 non-null   object
 1   1       8704 non-null   object
dtypes: object(2)
memory usage: 136.1+ KB


In [156]:
test_df = pd.read_csv(test_path, sep = '\t', header = None)
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2032 entries, 0 to 2031
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       2032 non-null   object
 1   1       2032 non-null   object
dtypes: object(2)
memory usage: 31.9+ KB


In [11]:
train_df.head()

,0,1
0,and,और
1,was,था
2,was,थी
3,for,लिये
4,that,उस


In [176]:
# trainPairs = dict(train_df.values)
# testPairs = dict(test_df.values)
# trainPairs = {}
# testPairs = {}

# for e, h in train_df.values:
#     trainPairs[e] = h

# for e, h in test_df.values:
#     testPairs[e] = h

# Loading pre-trained Fasttext monolingual embeddings

In [15]:
# !pip install fasttext
import fasttext
import fasttext.util

In [19]:
hindi_embed = fasttext.load_model('Embeddings/wiki.hi.bin')
english_embed = fasttext.load_model('Embeddings/wiki.en.bin')

In [196]:
english_words = list(train_df[0]) + list(test_df[0])
hindi_words = list(train_df[1]) + list(test_df[1])

In [198]:
print(len(english_words), len(hindi_words))

10736 10736


In [184]:
# english_vectors = {word : english_embed.get_word_vector(word) for word in english_words}
# hindi_vectors = {word: hindi_embed.get_word_vector(word) for word in hindi_words}

english_vectors = [english_embed.get_word_vector(word) for word in english_words]
hindi_vectors = [hindi_embed.get_word_vector(word) for word in hindi_words]

In [188]:
### Dimension of embeddings

dim = len(list(english_vectors)[0])
dim

300

# Applying orthogonal Proscrustes method

In [341]:
from scipy.linalg import orthogonal_procrustes
from scipy.spatial import procrustes

In [261]:
english_matrix = np.empty((0, dim))
hindi_matrix = np.empty((0, dim))

for i in range(len(train_df)):
    print(english_words[i], hindi_words[i])
    english_matrix = np.vstack([english_matrix, english_vectors[i].reshape((1, 300))])
    hindi_matrix = np.vstack([hindi_matrix, hindi_vectors[i].reshape((1, 300))])

and और
was था
was थी
for लिये
that उस
that कि
with साथ
from से
from इससे
this ये
this यह
this इस
utc यूटीसी
utc utc
his उसकी
his उसका
his उसके
not नही
not नहीं
are हैं
talk बात
which जिससे
also भी
has रै
were यहूद
but परन्तु
but लेकिन
but लेकीन
but मगर
but लकिन
one एक
new नया
new नई
first प्रथम
first पहली
first पहले
first पहला
page पृष्ठ
page पेज
you आपको
you आप
you तुम
they उन्होंने
they वे
had था
article लेख
article आलेख
who जिसने
who कौन
who जो
all सभी
all सब
their उनकी
their इनकी
their उनका
their उनके
there वहाँ
there वहां
there वहा
there वहीँ
been महरूम
made बनाया
made बना
its इसका
its इसकी
its इसके
people लोग
people लोगों
may मई
may may
after बाद
other अन्य
should चाहिए
two दो
score अंक
score स्कोर
her उसे
her उसका
her उसकी
her उसके
would होता
more ज्यादा
more आदि
more अधिक
she वो
about परिचय
when कब
when जब
time वक़्त
time वक्त
time समय
team टीम
team टोली
american अमेरिकन
american अमेरिकी
american अमरीकी
such ऐसी
such ऐसे
discussion परिचर्चा
discussion चर्चा
links लिंक
links लिं

In [202]:
english_matrix.shape, hindi_matrix.shape

((8704, 300), (8704, 300))

In [357]:
W = procrustes(english_matrix, hindi_matrix)
# W = np.multiply((1 + 0.01), W) - np.multiply(0.01, W @ W.T @ W)

# R, A = orthogonal_procrustes(english_matrix, hindi_matrix)

In [359]:
len(W)

3

In [299]:
def translateWord(english_word, R, english_model, hindi_vectors):
# english_word = 'ping'

    english_vector = english_embed.get_word_vector(english_word)
    translation_vector = np.dot(english_vector, R)
    i = max(np.arange(0, len(hindi_vectors)-1), key=lambda i: np.dot(hindi_vectors[i], translation_vector))
    
    return hindi_words[i]

In [329]:
hindi_translated = []

for word in train_df[0]:
    hindi_translated.append(translateWord(word, R, english_embed, hindi_vectors))

In [331]:
trueHindi = list(train_df[1])

preds = []

for i in range(len(trueHindi)):
    if trueHindi[i] == hindi_translated[i]:
        preds.append(1)
    else:
        preds.append(0)

In [333]:
print(sum(preds)/len(preds))

0.31893382352941174


# Evaluating the Translations